In [ ]:
%pip install xlrd openpyxl scikit-learn tqdm torchviz

In [ ]:
import pandas as pd
import sklearn as sk
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import tqdm as tqdm

np.version.full_version

In [ ]:
hope_production_df = pd.read_excel('data.xlsx', sheet_name="HOPE PRODUCTION")
hope_storage_df = pd.read_excel('data.xlsx', sheet_name="HOPE STORAGE AFTER COOKING")

faith_production_df = pd.read_excel('data.xlsx', sheet_name="FAITH PRODUCTION")
faith_storage_df = pd.read_excel('data.xlsx', sheet_name="FAITH STORAGE AFTER COOKING")

hope_faith_df = pd.read_excel('data.xlsx', sheet_name="HOPE-FAITH PACKAGE WEIGHTS")
hope_faith_df.rename(columns={
    x: f"Sample {i + 1}" for i, x in 
    enumerate((x for x in hope_faith_df.columns if "Unnamed" in x))}, inplace=True)

hope_faith_df.columns

In [ ]:
hope_df = hope_storage_df.merge(hope_production_df, right_on=["BATCH no.", "PRODUCTION DATE"], left_on=["BATCH no.", "BATCH INTO STORAGE"])
faith_df = faith_storage_df.merge(faith_production_df, right_on=["BATCH no.", "PRODUCTION DATE"], left_on=["BATCH no.", "BATCH INTO STORAGE"])
hope_df["PRODUCT"] = 5409
faith_df["PRODUCT"] = 5030

HOPE_SHELF_LIFE = 28
FAITH_SHELF_LIFE = 30

hope_df["ESTIMATED EXPIRY"] = hope_df["BATCH INTO STORAGE"] + pd.Timedelta(days=HOPE_SHELF_LIFE)
faith_df["ESTIMATED EXPIRY"] = faith_df["BATCH INTO STORAGE"] + pd.Timedelta(days=FAITH_SHELF_LIFE)

hope_pre_df = hope_df.merge(
        hope_faith_df, left_on=["ESTIMATED EXPIRY", "PRODUCT"], right_on=["EXPIRY DATE", "PRODUCT"]
    )
faith_pre_df = faith_df.merge(
        hope_faith_df, left_on=["ESTIMATED EXPIRY", "PRODUCT"], right_on=["EXPIRY DATE", "PRODUCT"]
    )


pre_df = pd.concat([hope_pre_df, faith_pre_df])
display(pre_df.columns)
pre_df["PRODUCT AMOUNT EXPECTED"] = [1000 * row["BATCH WEIGHT (kg) AFTER COOKING"] / row["AVERAGE WEIGHT (g)"] for _, row in pre_df.iterrows()]

pre_df

In [ ]:
INPUT_SIZE = 5
OUTPUT_SIZE = 1

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__(self)
        INTERNAL = 7
        self.l1 = nn.Linear(INPUT_SIZE, INTERNAL)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(INTERNAL, INTERNAL)
        self.lrelu = nn.LeakyReLU()
        self.l3 = nn.Linear(INTERNAL, OUTPUT_SIZE)
    
    def forward(self, x):
        return self.l3(self.lrelu(self.l2(self.relu(self.l1(x)))))

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df = pd.DataFrame()

df['product'] = le.fit_transform(pre_df['PRODUCT'])

df['time_in_storage'] = [x / np.timedelta64(1, 'D') for x in (pre_df["BATCH OUT OF STORAGE"] - pre_df["BATCH INTO STORAGE"]).values]
df['input_amount'] = preprocessing.MinMaxScaler().fit_transform(pre_df["BATCH WEIGHT (kg) BEFORE COOKING"].values.reshape(-1, 1))
df['cooking_out__storage_in'] = preprocessing.MinMaxScaler().fit_transform(pre_df["BATCH WEIGHT (kg) AFTER COOKING"].values.reshape(-1, 1))
df['storage_out__packaging_in'] = preprocessing.MinMaxScaler().fit_transform(pre_df["BATCH WEIGHT LEAVING STORAGE (KG)"].values.reshape(-1, 1))
df['product_amount_expected'] = preprocessing.MinMaxScaler().fit_transform(pre_df["PRODUCT AMOUNT EXPECTED"].values.reshape(-1, 1))

df

In [ ]:
class Data(torch.utils.data.Dataset):
    def __init__(self):
        self.data = df
        self.target = ["input_amount"]
    
    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index) -> tuple[torch.Tensor, torch.Tensor]:
        row = self.data.iloc[index]
        x = []
        y = []
        for k, v in row.items():
            if k in self.target:
                y.append(v)
            else:
                x.append(v)
        return tuple(map(torch.Tensor, [x, y]))

data = Data()
data[0]

In [ ]:
INPUT_SIZE = len(data[0][0])
OUTPUT_SIZE = len(data[0][1])
INTERNAL = 7

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(INPUT_SIZE, INTERNAL)
        self.lrelu = nn.LeakyReLU()
        self.l2 = nn.Linear(INTERNAL, INTERNAL)
        self.relu = nn.ReLU()
        self.l3 = nn.Linear(INTERNAL, OUTPUT_SIZE)
    
    def forward(self, x):
        return self.l3(self.relu(self.l2((self.lrelu(self.l1(x))))))

INPUT_SIZE, INTERNAL, OUTPUT_SIZE

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(data, [.8, .2])

display(train_dataset)
display(test_dataset)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device.type

In [ ]:
model = Net().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scaler = torch.amp.GradScaler(device.type)
epochs = 10

In [ ]:
for i in tqdm.tqdm(range(epochs)):
    for j, (x_train, y_train) in enumerate(train_dataset):
        x_train = x_train.to(device)
        y_train = y_train.to(device)

        with torch.amp.autocast(device_type=device.type):
            y_pred = model(x_train)
            cost = criterion(y_pred, y_train)
        
        optimizer.zero_grad(set_to_none=True)
        scaler.scale(cost).backward()
        scaler.step(optimizer)
        scaler.update()
    if i % 1 == 0:
        torch.save(model, f"model-{j}.pt")

torch.save(model, "model.pt")